## WebScrape MyAnime.net


Webscraping myanime.net for an updated anime list to be used for the animerecommender app

In [1]:
import urllib
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [9]:
url = "https://myanimelist.net/topanime.php?limit=0"

In [10]:
html = requests.get(url)
soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")

In [12]:
results_ = soup.find_all(class_= "ranking-list")
len(results_)

50

In [34]:
results_[1]

<tr class="ranking-list">
<td class="rank ac" valign="top">
<span class="lightLink top-anime-rank-text rank1">2</span>
</td>
<td class="title al va-t word-break">
<a class="hoverinfo_trigger fl-l ml12 mr8" href="https://myanimelist.net/anime/9253/Steins_Gate" id="#area9253" rel="#info9253">
<img alt="Anime: Steins;Gate" border="0" class="lazyload" data-src="https://cdn.myanimelist.net/r/50x70/images/anime/5/73199.jpg?s=97b97d568f25a02cf5a22dda13b5371f" data-srcset="https://cdn.myanimelist.net/r/50x70/images/anime/5/73199.jpg?s=97b97d568f25a02cf5a22dda13b5371f 1x, https://cdn.myanimelist.net/r/100x140/images/anime/5/73199.jpg?s=8fe506c27a2eba32611561a2dd116389 2x" height="70" width="50"/>
</a>
<div class="detail"><div id="area9253">
<div class="hoverinfo" id="info9253" rel="a9253"></div>
</div>
<div class="di-ib clearfix"><a class="hoverinfo_trigger fl-l fs14 fw-b" href="https://myanimelist.net/anime/9253/Steins_Gate" id="#area9253" rel="#info9253">Steins;Gate</a><a class="icon-watch ml

For the classifier we need:
1. The Anime name
2. The Genres
3. The ratings/score
4. The number of scored users: the number of members that have scored tis anime
5. Number of episodes
6. Number of members: the number of members that have added this anime
7. Type: i.e. TV, Movie, OVA

The most information can be obtained from href="https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood" in the class "hoverinfo_trigger fl-l ml12 mr8", but we see that a cuple can be retrieved from within the class "information di-ib mt4"

In [140]:
results_[0].find(class_="information di-ib mt4")

<div class="information di-ib mt4">
        TV (64 eps)<br/>
        Apr 2009 - Jul 2010<br/>
        1,777,286 members
      </div>

In [142]:
results_[0].find(class_="information di-ib mt4").text.strip()

'TV (64 eps)\n        Apr 2009 - Jul 2010\n        1,777,286 members'

We can extract four info: 
1. the type: here it is TV
2. The number of episodes: 64
3. Dates
4. the number of members

In [182]:
Type, Dates, members = results_[0].find(class_="information di-ib mt4").text.strip().splitlines()

In [193]:
float("".join(members.split()[0].split(",")))

1777286.0

Extract the number of episodes

In [159]:
Type

'TV (64 eps)'

In [167]:
[Type_, eps, n] = [", ".join(x.split()) for x in re.split(r'[()]',Type)]

In [177]:
eps = float(eps.split(",")[0])

In [41]:
results_[0].find(class_="hoverinfo_trigger fl-l fs14 fw-b")["href"]

'https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood'

In [42]:
url_= results_[0].find(class_="hoverinfo_trigger fl-l fs14 fw-b")["href"]
html_ = requests.get(url_)
soup_ = BeautifulSoup(html_.content, 'html.parser', from_encoding="utf-8")

In [150]:
soup_


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
    "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<link crossorigin="anonymous" href="//fonts.gstatic.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//fonts.googleapis.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//tags-cdn.deployads.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//www.googletagservices.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//www.googletagmanager.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//apis.google.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//pixel-sync.sitescout.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//pixel.tapad.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//c.deployads.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//requal-alleased.com/" rel="pr

In [151]:
# scrolling through this long list we see the tite is in class= "h1-title"
soup_.find(class_="h1-title").text.strip()

'Fullmetal Alchemist: Brotherhood'

Let's see we have: title, the number of member, the type, number of episodes, the dates. We nned to get the genres, score/ratings and the number of scored members. I think we will find all these in the borderClass.

In [71]:
soup_.find(class_="borderClass")

<td class="borderClass" style="border-width: 0 1px 0 0;" valign="top" width="225"><div style="width: 225px">
<div style="text-align: center;">
<a href="https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood/pics">
<img alt="Fullmetal Alchemist: Brotherhood" class="ac" data-src="https://cdn.myanimelist.net/images/anime/1223/96541.jpg" itemprop="image"/>
</a>
</div>
<!-- My List -->
<div class="profileRows pb0">
<a href="https://myanimelist.net/login.php?from=%2Fanime%2F5114%2FFullmetal_Alchemist__Brotherhood&amp;error=login_required">Add to My List</a>
</div>
<div class="addtolist-block js-anime-addtolist-block" id="addtolist" style="display: none;">
<input id="myinfo_anime_id" type="hidden" value="5114"/>
<input id="myinfo_curstatus" type="hidden" value=""/>
<span class="notice_open_public pb4">* Your list is public by default.</span>
<table border="0" cellpadding="0" cellspacing="0" width="100%">
<tr>
<td class="spaceit">Status:</td>
<td class="spaceit">
<select class="in

In [152]:
# list the genres from itemprop
soup_.find(class_="borderClass").find_all("span", itemprop="genre")

[<span itemprop="genre" style="display: none">Action</span>,
 <span itemprop="genre" style="display: none">Military</span>,
 <span itemprop="genre" style="display: none">Adventure</span>,
 <span itemprop="genre" style="display: none">Comedy</span>,
 <span itemprop="genre" style="display: none">Drama</span>,
 <span itemprop="genre" style="display: none">Magic</span>,
 <span itemprop="genre" style="display: none">Fantasy</span>,
 <span itemprop="genre" style="display: none">Shounen</span>]

In [89]:
genres = [genre.text.strip() for genre in soup_.find(class_="borderClass").find_all("span", itemprop="genre")]
genres

['Action',
 'Military',
 'Adventure',
 'Comedy',
 'Drama',
 'Magic',
 'Fantasy',
 'Shounen']

In [194]:
# the score
float(soup_.find(class_="borderClass").find_all("span", itemprop="ratingValue")[0].text.strip())

9.23

In [158]:
# number of scoring members: the number of members that have scored tis anime
float(soup_.find(class_="borderClass").find_all("span", itemprop="ratingCount")[0].text.strip())

1085943.0

### Puttting it all together

Nowwe have all the elemets, let's define a function to extract the info

In [208]:
def parse_MAl(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
    results = soup.find_all(class_= "ranking-list")
    
    df = pd.DataFrame(columns=["name","type","episodes","members","score_members", "rating","genre","dates"])
    i = 0
    for result in results:
        #print(i)
        url_= result.find(class_="hoverinfo_trigger fl-l fs14 fw-b")["href"]
        html_ = requests.get(url_)
        soup_ = BeautifulSoup(html_.content, 'html.parser', from_encoding="utf-8")
        
        try:
            name = soup_.find(class_="h1-title").text.strip()
        except:
            None
            
        Type, Dates, members = result.find(class_="information di-ib mt4").text.strip().splitlines()
        try:
            members = float("".join(members.split()[0].split(",")))
        except:
            None
            
        [Type_, eps, n] = [", ".join(x.split()) for x in re.split(r'[()]',Type)]
        
        try:
            eps = float(eps.split(",")[0])
        except:
            None
        
        try:
            genres = [genre.text.strip() for genre in soup_.find(class_="borderClass").find_all("span", itemprop="genre")]
            
        except:
            None
        
        try:
            score = float(soup_.find(class_="borderClass").find_all("span", itemprop="ratingValue")[0].text.strip())
        except:
            None
        try:
            score_members = float(soup_.find(class_="borderClass").find_all("span", itemprop="ratingCount")[0].text.strip())
        except:
            None
        
        df = df.append({
            "name": name,
            "type": Type_,
            "episodes": eps,
            "members": members,
            "score_members": score_members,
            "rating": score,
            "genre": genres,
            "dates": Dates
        },ignore_index=True)
        
        i+=1
    return df

In [209]:
url = "https://myanimelist.net/topanime.php?limit=0"

parse_MAl(url)

,name,type,episodes,members,score_members,rating,genre,dates
0,Fullmetal Alchemist: Brotherhood,TV,64.0,1777286.0,1085943.0,9.23,"[Action, Military, Adventure, Comedy, Drama, M...",Apr 2009 - Jul 2010
1,Steins;Gate,TV,24.0,1450929.0,796015.0,9.13,"[Thriller, Sci-Fi]",Apr 2011 - Sep 2011
2,Gintama°Gintama Season 4,TV,51.0,313855.0,110913.0,9.12,"[Action, Comedy, Historical, Parody, Samurai, ...",Apr 2015 - Mar 2016
3,Hunter x Hunter (2011)Hunter x Hunter,TV,148.0,1197308.0,679156.0,9.12,"[Action, Adventure, Fantasy, Shounen, Super Po...",Oct 2011 - Sep 2014
4,Ginga Eiyuu DensetsuLegend of the Galactic Heroes,OVA,110.0,189727.0,43203.0,9.11,"[Military, Sci-Fi, Space, Drama]",Jan 1988 - Mar 1997
5,Gintama'Gintama Season 2,TV,51.0,300658.0,126233.0,9.09,"[Action, Sci-Fi, Comedy, Historical, Parody, S...",Apr 2011 - Mar 2012
6,Shingeki no Kyojin Season 3 Part 2Attack on Ti...,TV,10.0,588666.0,359873.0,9.07,"[Action, Drama, Fantasy, Military, Mystery, Sh...",Apr 2019 - Jul 2019
7,Gintama': EnchousenGintama: Enchousen,TV,13.0,180871.0,90234.0,9.05,"[Action, Comedy, Historical, Parody, Samurai, ...",Oct 2012 - Mar 2013
8,Kimi no Na wa.Your Name.,Movie,1.0,1296770.0,867273.0,9.05,"[Romance, Supernatural, School, Drama]",Aug 2016 - Aug 2016
9,3-gatsu no Lion 2nd SeasonMarch Comes In Like ...,TV,22.0,197597.0,91893.0,9.03,"[Drama, Game, Seinen, Slice of Life]",Oct 2017 - Mar 2018


Success!!!! This is awesome. Now to extract all anime. To do so look at the url: url = "https://myanimelist.net/topanime.php?limit=0". The "limit=0" can be used to scrool through the list. From the website the limit is 16750, which we will set it to.

In [210]:
def webscrape_MAl(limit=16750):
    url_template = "https://myanimelist.net/topanime.php?limit={}"
    df = pd.DataFrame(columns=["name","type","episodes","members","score_members", "rating","genre","dates"])
    for start in range(0,limit, 50): # iterate in steps of 50
        url = url_template.format(start)
        df_temp = parse_MAl(url)
        df = df.append(df_temp, ignore_index=True)
    # save to disk
    
    df.to_csv('My_Anime_List_uncleaned.csv', encoding='utf-8')

In [211]:
#test with a smaller number
webscrape_MAl(200)

In [213]:
anime = pd.read_csv("My_Anime_List_uncleaned.csv")
anime.head()

,Unnamed: 0,name,type,episodes,members,score_members,rating,genre,dates
0,0,Fullmetal Alchemist: Brotherhood,TV,64.0,1777286.0,1085943.0,9.23,"['Action', 'Military', 'Adventure', 'Comedy', ...",Apr 2009 - Jul 2010
1,1,Steins;Gate,TV,24.0,1450929.0,796015.0,9.13,"['Thriller', 'Sci-Fi']",Apr 2011 - Sep 2011
2,2,Gintama°Gintama Season 4,TV,51.0,313855.0,110913.0,9.12,"['Action', 'Comedy', 'Historical', 'Parody', '...",Apr 2015 - Mar 2016
3,3,Hunter x Hunter (2011)Hunter x Hunter,TV,148.0,1197308.0,679156.0,9.12,"['Action', 'Adventure', 'Fantasy', 'Shounen', ...",Oct 2011 - Sep 2014
4,4,Ginga Eiyuu DensetsuLegend of the Galactic Heroes,OVA,110.0,189727.0,43203.0,9.11,"['Military', 'Sci-Fi', 'Space', 'Drama']",Jan 1988 - Mar 1997


In [214]:
anime.shape

(200, 9)

In [215]:
anime.tail()

,Unnamed: 0,name,type,episodes,members,score_members,rating,genre,dates
195,195,Youjo Senki Movie,Movie,1.0,50650.0,47923.0,8.35,"['Action', 'Military', 'Magic']",Apr 2002 - Apr 2002
196,196,Kono Subarashii Sekai ni Shukufuku wo! 2KonoSu...,TV,10.0,753044.0,475991.0,8.33,"['Adventure', 'Comedy', 'Parody', 'Supernatura...",Jan 2017 - Mar 2017
197,197,Natsume Yuujinchou Go Specials,Special,2.0,27123.0,11701.0,8.33,"['Slice of Life', 'Demons', 'Supernatural', 'D...",Mar 2017 - Apr 2017
198,198,Rurouni Kenshin: Meiji Kenkaku RomantanRurouni...,TV,94.0,346975.0,163579.0,8.33,"['Action', 'Adventure', 'Comedy', 'Historical'...",Jan 1996 - Sep 1998
199,199,Shokugeki no Souma: Ni no SaraFood Wars! The S...,TV,13.0,626307.0,383361.0,8.33,"['Ecchi', 'School', 'Shounen']",Jul 2016 - Sep 2016


Cool. Now to extract all the data, I'll add the functions to a script that I can conveniently run when i need to update.